In [1]:
# import tensorflow as tf
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from tensorflow.keras.models import load_model
# from multiprocessing import cpu_count
# from warnings import catch_warnings
# from warnings import filterwarnings
# from pandas import read_csv
# import matplotlib.pyplot as plt
# from pandas import DataFrame
# import time
# #from datetime import datetime
# import datetime
# import itertools
# import mysql.connector
# from mysql.connector import connection
# from mysql.connector import errorcode
# from statistics import mean
# import mysql.connector
# import requests
# from datetime import datetime
# import pytz
# import time
# import os
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from sklearn.preprocessing import OneHotEncoder
# from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error, accuracy_score

# import torch.nn.functional as F
# import torch
# import torch.nn as nn
# import torch.optim as optim

In [46]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from multiprocessing import cpu_count
from warnings import catch_warnings, filterwarnings
from pandas import read_csv, DataFrame
import time
import datetime
import itertools
import mysql.connector
from mysql.connector import connection, errorcode
from statistics import mean
import requests
import pytz
import os
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error, accuracy_score
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [30]:
def loadData(t):
    myconn = mysql.connector.connect(host="82.180.142.51",
                                     database="u978805288_land",
                                     user="u978805288_root",
                                     password="ACSL@b123")
    (myconn)

    sql_select_query = "select sensor_id, value from data_log, info_time where data_log.info_id = info_time.info_id AND data_log.triplet_id = '"+t+"' ORDER BY data_log.info_id desc limit 70"
    date_select_query = "select Date_Time from info_time where triplet_id='"+t+"' order by info_id desc limit 1"

    cursor = myconn.cursor()
    cursor.execute(sql_select_query)
    data = cursor.fetchall()

    cursor.execute(date_select_query)
    date = cursor.fetchall()

    myconn.close()
    return data, date[0][0]

In [31]:
# triplet_id = "t39"
# data, date_value = loadData(triplet_id)
# print("Data:")
# # print(data)
# for row in data:
#     print(row)

# print("Date:", date_value)

In [32]:
def dataPreprocess(data):
    result = []
    data.reverse()

    combined_data = []
    for i in range(0, len(data), 7):
        row = data[i:i+7]
        sensor_data = []
        for s, d in row:
            # print("R: ",row)
            sid = s
            if sid == 's6':
                continue
            elif sid == 's5':
                x = d.split(',')[:6]
                for j in x:
                    sensor_data.append(float(j))
                x = d.split(',')[-1]
                sensor_data.append(float(x))
            elif sid.startswith('s'):
                x = d.split(',')[:]
                for j in x:
                    sensor_data.append(float(j))

        combined_data.extend(sensor_data)

    data = np.array(combined_data)
    return combined_data


# preprocessed_data = dataPreprocess(data)
# split_rows = [preprocessed_data[i:i+13]
#               for i in range(0, len(preprocessed_data), 13)]

# for row in split_rows:
#     print(row)

In [33]:
# import tensorflow as tf
# data_list = []
# count = 0
# for i in range(10):
#     d = preprocessed_data[count:count+13]
#     data_list.append(d)
#     count += 13
# print(data_list)

# data_array = np.array(data_list)
# data_array = data_array.reshape((-1, 10, 13))
# data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)

In [34]:
# data_tensor.shape

In [35]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error, accuracy_score

import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim

In [36]:
# Define the LSTM model for multi-step prediction
class MultiStepWeatherLandslidePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_len, num_weather_params, num_classes):
        super(MultiStepWeatherLandslidePredictor, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers, batch_first=True)

        # Fully connected layers for weather parameters regression (10, 30, and 60 minutes ahead)
        self.fc_weather_10 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_weather_params)
        )
        self.fc_weather_30 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_weather_params)
        )
        self.fc_weather_60 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_weather_params)
        )

        # Fully connected layers for landslide classification (10, 30, and 60 minutes ahead)
        self.fc_landslide_10 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.fc_landslide_30 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.fc_landslide_60 = nn.Sequential(
            nn.Linear(hidden_size * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(
            0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(
            0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Flatten the output of LSTM to feed into fully connected layers
        out_flattened = out.reshape(out.shape[0], -1)

        # Predicting weather parameters
        weather_output_10 = self.fc_weather_10(out_flattened)
        weather_output_30 = self.fc_weather_30(out_flattened)
        weather_output_60 = self.fc_weather_60(out_flattened)

        # Predicting landslide classes
        landslide_output_10 = self.fc_landslide_10(out_flattened)
        landslide_output_30 = self.fc_landslide_30(out_flattened)
        landslide_output_60 = self.fc_landslide_60(out_flattened)

        return (weather_output_10, weather_output_30, weather_output_60,
                landslide_output_10, landslide_output_30, landslide_output_60)


# Instantiate the model
input_size = 13  # Exclude the first column
hidden_size = 64
num_layers = 2
seq_len = 10
num_weather_params = 6
num_classes = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiStepWeatherLandslidePredictor(
    input_size, hidden_size, num_layers, seq_len, num_weather_params, num_classes)
model = model.to(device)

In [37]:
# Load the model state dictionary
PATH = "model_weights.pth"

model = MultiStepWeatherLandslidePredictor(
    input_size, hidden_size, num_layers, seq_len, num_weather_params, num_classes)
model.load_state_dict(torch.load(PATH))
model.eval()
print(f"Entire model loaded from {PATH}")

Entire model loaded from model_weights.pth


In [38]:
# Database credentials
connection_config = {
    'host': "82.180.142.51",
    'database': "u978805288_land",
    'user': "u978805288_root",
    'password': "ACSL@b123"
}

In [39]:
import requests
import datetime

def insertDatabase(prediction, triplet_id):
    connection = mysql.connector.connect(**connection_config)
    
    try:
        (connection)

        # current_time = date + datetime.timedelta()
        current_time = datetime.datetime.now()  # Get the current time
        # current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
        next_time_10 = current_time + datetime.timedelta(minutes=10)
        next_time_10 = next_time_10.strftime('%Y-%m-%d %H:%M:%S')
        # print('current_time:',current_time)
        val_10 = "(NULL,'" + (triplet_id)+"'," + str(prediction[0][0])+"," + str(prediction[0][1])+","+str(prediction[0][2])+","+str(
            prediction[0][3])+","+str(prediction[0][4])+","+str(prediction[0][5]) + ",'"+str(next_time_10) + "','no')"
        add_prediction_10 = "INSERT INTO Prediction_Real_Data  VALUES "+val_10
        print(add_prediction_10)

        next_time_30 = current_time + datetime.timedelta(minutes=30)
        next_time_30 = next_time_30.strftime('%Y-%m-%d %H:%M:%S')
        # print('current_time:',current_time)
        val_30 = "(NULL,'" + (triplet_id)+"'," + str(prediction[1][0])+"," + str(prediction[1][1])+","+str(prediction[1][2])+","+str(
            prediction[1][3])+","+str(prediction[1][4])+","+str(prediction[1][5]) + ",'"+str(next_time_30) + "','no')"
        add_prediction_30 = "INSERT INTO Prediction_Real_Data  VALUES "+val_30
        print(add_prediction_30)

        next_time_60 = current_time + datetime.timedelta(minutes=60)
        next_time_60 = next_time_60.strftime('%Y-%m-%d %H:%M:%S')
        # print('current_time:',current_time)
        val_60 = "(NULL,'" + (triplet_id)+"'," + str(prediction[2][0])+"," + str(prediction[2][1])+","+str(prediction[2][2])+","+str(
            prediction[2][3])+","+str(prediction[2][4])+","+str(prediction[2][5]) + ",'"+str(next_time_60) + "','no')"
        add_prediction_60 = "INSERT INTO Prediction_Real_Data  VALUES "+val_60
        print(add_prediction_60)

        cursor = connection.cursor()
        cursor.execute(add_prediction_10)
        cursor.execute(add_prediction_30)
        cursor.execute(add_prediction_60)
        connection.commit()

        print(cursor.rowcount, "Record inserted successfully into prediction table")
        cursor.close()


    except mysql.connector.Error as error:
        print("Failed to insert record into prediction table {}".format(error))
        
    finally:
        if connection.is_connected():
            connection.close()

In [40]:
station = ["t39","t56","t61","t72","t79","t101","t103"]
# Location ID dictionary
location_id = {
    "t39": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak3
    "t56": {"lat": 31.590, "lon": 78.300},  # Purbani_kinnaur
    "t61": {"lat": 31.248, "lon": 77.089},  # Tattapani_Mandi
    "t72": {"lat": 32.226, "lon": 76.326},  # Dharmshala Kangra
    "t79": {"lat": 32.292, "lon": 75.996},  # Bariara_Nurpur_Kangra
    "t101": {"lat": 31.104, "lon": 77.173},  # shimla
    "t103": {"lat": 31.708, "lon": 76.932},  # mandi
}

#t38, t37, t39, t71

In [41]:
# OpenWeatherMap API key
API_KEY = 'bdc503040a4bbf2de4eeb9eb3ab841ed'

def fetch_weather_data(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return None
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
        return None
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"General error occurred: {req_err}")
        return None
    except ValueError as json_err:
        print(f"JSON decode error: {json_err}")
        return None

    try:
        timestamp = data['dt']
        temp_k = data['main']['temp']
        humidity = data['main']['humidity']
        pressure = data['main']['pressure']
        rain = data.get('rain', {}).get('1h', 0)
        clouds = data['clouds']['all']*100  # percentage to number

        temp_c = temp_k - 273.15
        temp_c = round(temp_c, 2)
        ist = pytz.timezone('Asia/Kolkata')
        datetime_ist = datetime.datetime.fromtimestamp(timestamp, ist)
        human_readable_date = datetime_ist.strftime('%Y-%m-%d %H:%M:%S')

        weather_info = {
            "datetime": human_readable_date,
            "temp": temp_c,
            "hum": humidity,
            "press": pressure,
            "rain": rain,
            "clouds": clouds
        }
    except KeyError as e:
        print(f"Missing data in the API response: {e}")
        return None

    return weather_info


def insertWeatherApiData(weather_data_api, triplet_id):
    if weather_data_api is None:
        print(
            f"Skipping database insert for {triplet_id} due to previous error.")
        return

    try:
        connection = mysql.connector.connect(**connection_config)

        cursor = connection.cursor()
        insert_query = """
        INSERT INTO weather_data_api (datetime, triplet_id, temp, hum, press, rain, light)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        record = (weather_data_api["datetime"], triplet_id, weather_data_api["temp"], weather_data_api["hum"],
                  weather_data_api["press"], weather_data_api["rain"], weather_data_api["clouds"])

        cursor.execute(insert_query, record)
        connection.commit()

        print(f"Weather API Record inserted successfully for {triplet_id}: {record}")
    except mysql.connector.Error as error:
        print(f"Failed to insert record: {error}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()


In [42]:
# # INSERTING WEATHER API DATA TO DATABASE
# while True:
#     for triplet_id, coords in location_id.items():
#         lat = coords["lat"]
#         lon = coords["lon"]
#         weather_data_api = fetch_weather_data(lat, lon)

#         # Print statement for debugging
#         # print(f"Weather data for {triplet_id}:")
#         # for key, value in weather_data_api.items():
#         #     print(f"{key}: {value}")
#         # print("\n")

#         # Function call to store data
#         insertWeatherApiData(weather_data_api, triplet_id)

#     print("Data fetched and printed. Waiting for 10 minutes before the next run.")
#     time.sleep(600)  # Sleep for 600 seconds (10 minutes)

In [43]:
def fetch_preprocessed_weather_data(triplet_id):
    try:
        connection = mysql.connector.connect(**connection_config)
        cursor = connection.cursor()
        query = """
        SELECT * FROM weather_data_api
        WHERE triplet_id = %s
        ORDER BY datetime DESC
        LIMIT 10
        """
        cursor.execute(query, (triplet_id,))
        rows = cursor.fetchall()
        fetched_data = []

        for row in rows:
            float_row = []
            for elem in row:
                # Check if element is numeric (excluding datetime objects)
                if isinstance(elem, (int, float)):
                    float_row.append(float(elem))
                elif isinstance(elem, str) and elem.replace('.', '', 1).isdigit():
                    float_row.append(float(elem))
            fetched_data.append(float_row[-5:])
            # print(float_row[-5:])  # Print the last 5 elements of the row

        # Convert fetched_data to a NumPy array with dtype=np.float32
        fetched_data = np.array(fetched_data, dtype=np.float32)

        return fetched_data
    except mysql.connector.Error as error:
        print(f"Failed to fetch records: {error}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [48]:
# location_id = {
#     "t39": {"lat": 31.777, "lon": 76.971}  # Griffon_Peak3
    
# }

location_id = {
    "t39": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak3
    "t56": {"lat": 31.590, "lon": 78.300},  # Purbani_kinnaur
    "t61": {"lat": 31.248, "lon": 77.089},  # Tattapani_Mandi
    "t72": {"lat": 32.226, "lon": 76.326},  # Dharmshala Kangra
    "t79": {"lat": 32.292, "lon": 75.996},  # Bariara_Nurpur_Kangra
    "t101": {"lat": 31.104, "lon": 77.173},  # shimla
    "t103": {"lat": 31.708, "lon": 76.932},  # mandi
}

In [49]:
while True:
    for t_id, coords in location_id.items():
        data, date = loadData(t_id)
        print(date)
        date = date + timedelta(minutes=10)
        preprocessed_data = dataPreprocess(data)
        data_list = []
        count = 0
        for i in range(10):
            d = preprocessed_data[count:count+13]
            data_list.append(d)
            count += 13

        data_array = np.array(data_list)
        
        # data_torch = torch.tensor(data_array) 
        # print(data_torch)
        
        lat = coords["lat"]
        lon = coords["lon"]
        weather_data_api = fetch_weather_data(lat, lon)

        # Print statement for debugging
        # print(f"Weather data for {triplet_id}:")
        # for key, value in weather_data_api.items():
        #     print(f"{key}: {value}")
        # print("\n")

        # Function call to store data
        insertWeatherApiData(weather_data_api, t_id)

        print("Data fetched and printed. Waiting for 10 minutes before the next run.")
        data_array = data_array.reshape((-1, 10, 13)).astype(np.float32)
        weather_data = fetch_preprocessed_weather_data(t_id)
        for i in range(10):
            for j in range(5):
                data_array[0][0][j] = weather_data[0][j]

            # print("UPDATED:\n")
            # print(data_array)

        # Convert to PyTorch tensor
        print(weather_data)
        data_torch = torch.tensor(data_array)  
        
        with torch.no_grad():
            (weather_output_10, weather_output_30, weather_output_60,
                landslide_output_10, landslide_output_30, landslide_output_60) = model(data_torch)

            # Convert outputs to numpy arrays and move to CPU
            weather_output_10 = weather_output_10.cpu().numpy()
            weather_output_30 = weather_output_30.cpu().numpy()
            weather_output_60 = weather_output_60.cpu().numpy()
            landslide_output_10 = torch.argmax(
                landslide_output_10, dim=1).cpu().numpy()
            landslide_output_30 = torch.argmax(
                landslide_output_30, dim=1).cpu().numpy()
            landslide_output_60 = torch.argmax(
                landslide_output_60, dim=1).cpu().numpy()

        formatted_data_format_10 = [
            ["{:.2f}".format(num) for num in row] for row in weather_output_10]
        # print("Formatted Data (format method):", formatted_data_format_10)

        # print("Predicted weather parameters for 30 minutes ahead:", weather_output_30)
        formatted_data_format_30 = [
            ["{:.2f}".format(num) for num in row] for row in weather_output_30]
        # print("Formatted Data (format method):", formatted_data_format_30)

        # print("Predicted weather parameters for 60 minutes ahead:", weather_output_60)
        formatted_data_format_60 = [
            ["{:.2f}".format(num) for num in row] for row in weather_output_60]
        # print("Formatted Data (format method):", formatted_data_format_60)

        prediction = []
        prediction.append(formatted_data_format_10[0])
        prediction.append(formatted_data_format_30[0])
        prediction.append(formatted_data_format_60[0])

        prediction[0][5] = landslide_output_10[0]
        prediction[1][5] = landslide_output_30[0]
        prediction[2][5] = landslide_output_60[0]
        # print("Predicted landslide class for 10 minutes ahead:", landslide_output_10)
        # print("Predicted landslide class for 30 minutes ahead:", landslide_output_30)
        # print("Predicted landslide class for 60 minutes ahead:", landslide_output_60)

        # print("Prediction: ", prediction[0])
        insertDatabase(prediction, t_id)
        print('Enjoy')
    time.sleep(600)

2024-06-10 21:05:16
Weather API Record inserted successfully for t39: ('2024-06-10 21:02:53', 't39', 23.15, 24, 1007, 0, 9700)
Data fetched and printed. Waiting for 10 minutes before the next run.
[[   23.15    24.    1007.       0.    9700.  ]
 [   23.15    24.    1007.       0.    9700.  ]
 [   23.69    21.    1006.       0.    6500.  ]
 [   29.83    15.    1003.       0.   10000.  ]
 [   21.74    40.    1006.       0.    2300.  ]
 [   21.74    40.    1006.       0.    2300.  ]
 [   21.74    40.    1006.       0.    2300.  ]
 [   26.18    30.    1005.       0.    4100.  ]
 [   28.38    21.    1003.       0.     300.  ]
 [   28.38    21.    1003.       0.     300.  ]]
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t39',24.15,71.49,840.04,-0.54,3194.09,0,'2024-06-10 21:16:07','no')
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t39',23.24,71.36,851.33,-1.27,2870.31,0,'2024-06-10 21:36:07','no')
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t39',22.10,72.44,853.07,-0.46,2642.89,0,'2